# FaissStore

```{=mdx}

:::tip Compatibility
Only available on Node.js.
:::

```

[Faiss](https://github.com/facebookresearch/faiss) is a library for efficient similarity search and clustering of dense vectors.

LangChain.js supports using Faiss as a locally-running vectorstore that can be saved to a file. It also provides the ability to read the saved file from the [LangChain Python implementation](https://python.langchain.com/docs/integrations/vectorstores/faiss#saving-and-loading).

This guide provides a quick overview for getting started with Faiss [vector stores](/docs/concepts/#vectorstores). For detailed documentation of all `FaissStore` features and configurations head to the [API reference](https://api.js.langchain.com/classes/langchain_community_vectorstores_faiss.FaissStore.html).

## Overview

### Integration details

| Class | Package | [PY support](https://python.langchain.com/docs/integrations/vectorstores/faiss) |  Package latest |
| :--- | :--- | :---: | :---: |
| [`FaissStore`](https://api.js.langchain.com/classes/langchain_community_vectorstores_faiss.FaissStore.html) | [`@langchain/community`](https://npmjs.com/@langchain/community) | ✅ |  ![NPM - Version](https://img.shields.io/npm/v/@langchain/community?style=flat-square&label=%20&) |

## Setup

To use Faiss vector stores, you'll need to install the `@langchain/community` integration package and the [`faiss-node`](https://github.com/ewfian/faiss-node) package as a peer dependency.

This guide will also use [OpenAI embeddings](/docs/integrations/text_embedding/openai), which require you to install the `@langchain/openai` integration package. You can also use [other supported embeddings models](/docs/integrations/text_embedding) if you wish.

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/community faiss-node @langchain/openai @langchain/core
</Npm2Yarn>
```

### Credentials

Because Faiss runs locally, you do not need any credentials to use it.

If you are using OpenAI embeddings for this guide, you'll need to set your OpenAI key as well:

```typescript
process.env.OPENAI_API_KEY = "YOUR_API_KEY";
```

If you want to get automated tracing of your model calls you can also set your [LangSmith](https://docs.smith.langchain.com/) API key by uncommenting below:

```typescript
// process.env.LANGCHAIN_TRACING_V2="true"
// process.env.LANGCHAIN_API_KEY="your-api-key"
```

## Instantiation

In [2]:
import { FaissStore } from "@langchain/community/vectorstores/faiss";
import { OpenAIEmbeddings } from "@langchain/openai";

const embeddings = new OpenAIEmbeddings({
  model: "text-embedding-3-small",
});

const vectorStore = new FaissStore(embeddings, {});

## Manage vector store

### Add items to vector store

In [3]:
import type { Document } from "@langchain/core/documents";

const document1: Document = {
  pageContent: "The powerhouse of the cell is the mitochondria",
  metadata: { source: "https://example.com" }
};

const document2: Document = {
  pageContent: "Buildings are made out of brick",
  metadata: { source: "https://example.com" }
};

const document3: Document = {
  pageContent: "Mitochondria are made out of lipids",
  metadata: { source: "https://example.com" }
};

const document4: Document = {
  pageContent: "The 2024 Olympics are in Paris",
  metadata: { source: "https://example.com" }
}

const documents = [document1, document2, document3, document4];

await vectorStore.addDocuments(documents, { ids: ["1", "2", "3", "4"] });

[ '1', '2', '3', '4' ]


### Delete items from vector store

In [4]:
await vectorStore.delete({ ids: ["4"] });

## Query vector store

Once your vector store has been created and the relevant documents have been added you will most likely wish to query it during the running of your chain or agent. 

### Query directly

Performing a simple similarity search can be done as follows:

In [5]:
const similaritySearchResults = await vectorStore.similaritySearch("biology", 2);

for (const doc of similaritySearchResults) {
  console.log(`* ${doc.pageContent} [${JSON.stringify(doc.metadata, null)}]`);
}

* The powerhouse of the cell is the mitochondria [{"source":"https://example.com"}]
* Mitochondria are made out of lipids [{"source":"https://example.com"}]


Filtering by metadata is currently not supported.

If you want to execute a similarity search and receive the corresponding scores you can run:

In [6]:
const similaritySearchWithScoreResults = await vectorStore.similaritySearchWithScore("biology", 2);

for (const [doc, score] of similaritySearchWithScoreResults) {
  console.log(`* [SIM=${score.toFixed(3)}] ${doc.pageContent} [${JSON.stringify(doc.metadata)}]`);
}

* [SIM=1.671] The powerhouse of the cell is the mitochondria [{"source":"https://example.com"}]
* [SIM=1.705] Mitochondria are made out of lipids [{"source":"https://example.com"}]


### Query by turning into retriever

You can also transform the vector store into a [retriever](/docs/concepts/retrievers) for easier usage in your chains. 

In [7]:
const retriever = vectorStore.asRetriever({
  k: 2,
});
await retriever.invoke("biology");

[
  {
    pageContent: 'The powerhouse of the cell is the mitochondria',
    metadata: { source: 'https://example.com' }
  },
  {
    pageContent: 'Mitochondria are made out of lipids',
    metadata: { source: 'https://example.com' }
  }
]


### Usage for retrieval-augmented generation

For guides on how to use this vector store for retrieval-augmented generation (RAG), see the following sections:

- [Tutorials: working with external knowledge](/docs/tutorials/#working-with-external-knowledge).
- [How-to: Question and answer with RAG](/docs/how_to/#qa-with-rag)
- [Retrieval conceptual docs](/docs/concepts/retrieval)

## Merging indexes

Faiss also supports merging existing indexes:

In [ ]:
// Create an initial vector store
const initialStore = await FaissStore.fromTexts(
  ["Hello world", "Bye bye", "hello nice world"],
  [{ id: 2 }, { id: 1 }, { id: 3 }],
  new OpenAIEmbeddings()
);

// Create another vector store from texts
const newStore = await FaissStore.fromTexts(
  ["Some text"],
  [{ id: 1 }],
  new OpenAIEmbeddings()
);

// merge the first vector store into vectorStore2
await newStore.mergeFrom(initialStore);

// You can also create a new vector store from another FaissStore index
const newStore2 = await FaissStore.fromIndex(
  newStore,
  new OpenAIEmbeddings()
);

await newStore2.similaritySearch("Bye bye", 1);

## Save an index to file and load it again

To persist an index on disk, use the `.save` and static `.load` methods:

In [ ]:
// Create a vector store through any method, here from texts as an example
const persistentStore = await FaissStore.fromTexts(
  ["Hello world", "Bye bye", "hello nice world"],
  [{ id: 2 }, { id: 1 }, { id: 3 }],
  new OpenAIEmbeddings()
);

// Save the vector store to a directory
const directory = "your/directory/here";

await persistentStore.save(directory);

// Load the vector store from the same directory
const loadedVectorStore = await FaissStore.load(
  directory,
  new OpenAIEmbeddings()
);

// vectorStore and loadedVectorStore are identical
const result = await loadedVectorStore.similaritySearch("hello world", 1);
console.log(result);

## Reading saved files from Python

To enable the ability to read the saved file from [LangChain Python's implementation](https://python.langchain.com/docs/integrations/vectorstores/faiss#saving-and-loading), you'll need to install the [`pickleparser`](https://github.com/ewfian/pickleparser) package.

```{=mdx}
<Npm2Yarn>
  pickleparser
</Npm2Yarn>
```

Then you can use the `.loadFromPython` static method:

In [ ]:
// The directory of data saved from Python
const directoryWithSavedPythonStore = "your/directory/here";

// Load the vector store from the directory
const pythonLoadedStore = await FaissStore.loadFromPython(
  directoryWithSavedPythonStore,
  new OpenAIEmbeddings()
);

// Search for the most similar document
await pythonLoadedStore.similaritySearch("test", 2);

## API reference

For detailed documentation of all `FaissStore` features and configurations head to the [API reference](https://api.js.langchain.com/classes/langchain_community_vectorstores_faiss.FaissStore.html)